# Initialization

In [1]:
import logging, rx
from rx import operators
from rx.subject import Subject
from threading import current_thread, Lock, Condition
from datetime import datetime, timedelta
from typing import Callable, Any, List

logger = logging.getLogger()
logger.addHandler(logging.StreamHandler())
logger.setLevel(logging.DEBUG)

def logWithTime(msg):
    logger.info(f'{datetime.now().strftime("%H:%M:%S.%f")} [{current_thread().ident:05d}] {msg}')

# Create observable functions

In [120]:
def sequenceInTime(period, elapsed) -> rx.Observable:
    observable = rx.interval(period).pipe(
        operators.map(lambda x: {
            'idx': x,
            'ts': datetime.now()
            }),
        operators.take_with_time(elapsed),
        # operators.observe_on(rx.scheduler.ThreadPoolScheduler(1)),
    )
    return observable

# Subscribe and operate

In [130]:
period = timedelta(seconds=.0)
elapsed = timedelta(seconds=10)
frequency = timedelta(seconds=1)
now = datetime.now()
startAt = datetime(year=now.year, month=now.month, day=now.day, hour=now.hour, minute=now.minute, second=now.second)
boundary = startAt + frequency
lastTickTime = None
windows = []
buffer = []
cv = Condition()
scheduler = rx.scheduler.ThreadPoolScheduler(1)
groupedTicks = {}

def onGroup(observable: rx.core.GroupedObservable):
    k = observable.key
    g = []
    def onGroupExpired():
        global groupedTicks, cv
        logWithTime(f'[{k:%H:%M:%S.%f}] [{len(g):5d}] ({g[0]["idx"]:5d}){g[0]["ts"]:%H:%M:%S.%f} .. ({g[-1]["idx"]:5d}){g[-1]["ts"]:%H:%M:%S.%f}')
        cv.acquire()
        groupedTicks[k] = g
        cv.release()
    observable.subscribe(
        on_next=lambda x: g.append(x),
        on_completed=onGroupExpired
    )

def onCompleted():
    cv.acquire()
    logWithTime('stream completed')
    cv.notify()
    cv.release()

logWithTime(f'start receive ticks')
with sequenceInTime(period, elapsed).pipe(
    operators.group_by_until(
        lambda x: datetime(x['ts'].year,
        x['ts'].month,
        x['ts'].day,
        x['ts'].hour,
        x['ts'].minute,
        x['ts'].second,# - (x['ts'].second % 3)
        # (int(f"{x['ts'].microsecond:06d}"[0:3]) - (int(f"{x['ts'].microsecond:06d}"[0:3]) % 500)) * 1000
        ),
        None,
        lambda g: rx.timer(1)
    ),
    operators.observe_on(scheduler),
    ).subscribe(
        on_next=lambda x: onGroup(x),
        on_completed=onCompleted,
    ) as subscriber:
    cv.acquire()
    if not cv.wait((elapsed + timedelta(seconds=1)).total_seconds()):
        logWithTime('timeout')
    logWithTime(f'{len([v for k in groupedTicks for v in groupedTicks[k]])} ticks received totally, in {len(groupedTicks)} groups')

logWithTime(f'{datetime.now() - now} elapsed')
cv.release()

23:39:32.772528 [15932] start receive ticks
23:39:33.785529 [17452] [23:39:32.000000] [  871] (    0)23:39:32.775533 .. (  870)23:39:32.999532
23:39:34.016532 [17420] [23:39:33.000000] [ 3895] (  871)23:39:33.000532 .. ( 4765)23:39:33.999531
23:39:35.011047 [08892] [23:39:34.000000] [ 3515] ( 4766)23:39:34.000532 .. ( 8280)23:39:34.999046
23:39:36.005044 [12784] [23:39:35.000000] [ 2553] ( 8281)23:39:35.000047 .. (10833)23:39:35.999045
23:39:37.003045 [17616] [23:39:36.000000] [ 3833] (10834)23:39:36.000046 .. (14666)23:39:36.999046
23:39:38.001045 [12620] [23:39:37.000000] [ 3940] (14667)23:39:37.000048 .. (18606)23:39:37.999047
23:39:39.014046 [16936] [23:39:38.000000] [ 3916] (18607)23:39:38.000047 .. (22522)23:39:38.999049
23:39:40.007046 [12672] [23:39:39.000000] [ 3629] (22523)23:39:39.000048 .. (26151)23:39:39.999045
23:39:41.004047 [14044] [23:39:40.000000] [ 3674] (26152)23:39:40.000046 .. (29825)23:39:40.999045
23:39:42.015045 [17288] [23:39:41.000000] [ 3723] (29826)23:39:41